<a href="https://colab.research.google.com/github/Allen123321/DEMO-DL/blob/master/HMM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np

infinite = float(-2 ** 31)


def log_sum_exp(a):
    """
    可以参考numpy中的log sum exp的API
    scipy.misc.logsumexp
    :param a:
    :return:
    """
    a = np.asarray(a)
    a_max = max(a)
    tmp = 0
    for k in a:
        tmp += math.exp(k - a_max)
    return a_max + math.log(tmp)


def convert_obs_seq_2_index(Q, index=None):
    """
    将观测序列转换为观测值的索引值
    Q:是输入的观测序列
    """
    if index is not None:
        cht = Q[index]
        if "黑" == cht:
            return 1
        else:
            return 0
    else:
        result = []
        for q in Q:
            if "黑" == q:
                result.append(1)
            else:
                result.append(0)
        return result


In [2]:
import numpy as np
def calc_alpha(pi, A, B, Q, alpha, fetch_index_by_obs_seq=None):
    """
    计算前向概率α的值
    :param pi: 初始的状态随机概率矩阵, n*1 => 经过对数转换
    :param A:  状态转移矩阵, n*n => 经过对数转换
    :param B:  状态和观测值之间的转移矩阵, n*m => 经过对数转换
    :param Q:  观测值序列, 长度为T
    :param alpha:  前向概率矩阵
    :param fetch_index_by_obs_seq: 根据序列获取对应的索引值，可以为None；是一个接受两个参数的函数，第一个参数为序列，第二个参数为索引值
    :return:  返回alpha矩阵同时也更新传入的alpha矩阵参数
    """
    # 1. 初始化
    fetch_index_by_obs_seq_f = fetch_index_by_obs_seq
    if fetch_index_by_obs_seq_f is None:
        # 默认是使用ord函数将对应位置的字符转换为ASCII码，eg: ord('a')=97; ord('中')=20013
        fetch_index_by_obs_seq_f = lambda obs, obs_index: ord(obs[obs_index])

    # 2. 获取相关变量
    n = np.shape(A)[0]
    T = len(Q)

    # 3. 更新t=1时刻(初始时刻)对应的前向概率值
    for i in range(n):
        alpha[0][i] = pi[i] + B[i][fetch_index_by_obs_seq_f(Q, 0)]

    # 4. 更新t=2,3....T时刻对应的前向概率值
    tmp = [0 for i in range(n)]
    for t in range(1, T):
        # TODO: 开始更新时刻t、状态为i的前向概率值
        for i in range(n):
            # 4.1. 计算累加值
            for j in range(n):
                tmp[j] = alpha[t - 1][j] + A[j][i]

            # 4.2. 计算log_sum_exp的值
            alpha[t][i] = log_sum_exp(tmp)

            # 4.3. 累加状态和观测值之间的转移矩阵
            alpha[t][i] += B[i][fetch_index_by_obs_seq_f(Q, t)]

    # 5. 返回最终返回值
    return alpha


def calc_beta(pi, A, B, Q, beta, fetch_index_by_obs_seq=None):
    """
    计算后向概率β的值
    :param pi: 初始的状态随机概率矩阵, n*1 => 经过对数转换
    :param A:  状态转移矩阵, n*n => 经过对数转换
    :param B:  状态和观测值之间的转移矩阵, n*m => 经过对数转换
    :param Q:  观测值序列, 长度为T
    :param beta: 后向概率矩阵
    :param fetch_index_by_obs_seq: 根据序列获取对应的索引值，可以为None；是一个接受两个参数的函数，第一个参数为序列，第二个参数为索引值
    :return:  返回beta矩阵同时也更新传入的beta矩阵参数
    """
    # 1. 初始化
    fetch_index_by_obs_seq_f = fetch_index_by_obs_seq
    if fetch_index_by_obs_seq_f is None:
        # 默认是使用ord函数将对应位置的字符转换为ASCII码，eg: ord('a')=97; ord('中')=20013
        fetch_index_by_obs_seq_f = lambda obs, obs_index: ord(obs[obs_index])

    # 2. 获取相关变量
    n = np.shape(A)[0]
    T = len(Q)

    # 3. 更新t=T时刻(初始时刻)对应的后向概率值
    for i in range(n):
        beta[T - 1][i] = 0

    # 4. 更新t=T-1,T-2....1时刻对应的前向概率值
    tmp = [0 for i in range(n)]
    for t in range(T - 2, -1, -1):
        # TODO: 更新时刻t对应状态为i的后向概率
        for i in range(n):
            # 4.1 计算累加值
            for j in range(n):
                tmp[j] = A[i][j] + beta[t + 1][j] + B[j][fetch_index_by_obs_seq_f(Q, t + 1)]

            # 4.2 计算log_sum_exp的值
            beta[t][i] = log_sum_exp(tmp)

    # 5. 结果返回
    return beta


def calc_gamma(alpha, beta, gamma):
    """
    根据alphe和beta的值计算gamma值
    最终结果保存在gamma矩阵中
    :param alpha: 传入的alpha矩阵 => 经过log转换后的
    :param beta:  传入的beta矩阵 => 经过log转换后的
    :param gamma: 传入的gamma矩阵，需要进行更新，最终结果是经过log转换后的
    :return: gamma矩阵
    """
    # 1. 获取相关变量
    T, n = np.shape(alpha)

    # 2. 遍历更新
    for t in range(T):
        # 2.1. 累加alpha和beta值(ppt上分子部分)
        for i in range(n):
            gamma[t][i] = alpha[t][i] + beta[t][i]

        # 2.2. 计算log_sum_exp的值（ppt上分母部分）
        lse = log_sum_exp(gamma[t])

        # 2.3. 计算最终结果
        for i in range(n):
            gamma[t][i] -= lse

    # 3. 返回最终结果
    return gamma


def calc_ksi(alpha, beta, A, B, Q, ksi, fetch_index_by_obs_seq=None):
    """
    计算时刻t的时候状态为i，时刻t+1的时候状态为j的联合概率ksi
    :param alpha:  传入的alpha矩阵 => 经过log转换后的
    :param beta:  传入的beta矩阵 => 经过log转换后的
    :param A: 状态转移矩阵, n*n => 经过对数转换
    :param B:  状态和观测值之间的转移矩阵, n*m => 经过对数转换
    :param Q:  观测值序列, 长度为T
    :param ksi: 待求解的ksi概率矩阵，最终结果是需要经过log转换的
    :param fetch_index_by_obs_seq: 根据序列获取对应的索引值，可以为None；是一个接受两个参数的函数，第一个参数为序列，第二个参数为索引值
    :return: 返回ksi矩阵
    """
    # 1. 初始化
    # 初始化序列转换为索引的方法
    fetch_index_by_obs_seq_f = fetch_index_by_obs_seq
    if fetch_index_by_obs_seq_f is None:
        fetch_index_by_obs_seq_f = lambda obs, obs_index: ord(obs[obs_index])

    # 2. 变量获取
    T, n = np.shape(alpha)

    # 3. 开始迭代更新ksi矩阵
    tmp = np.zeros((n, n))
    for t in range(T - 1):
        # 3.1 计算t时刻状态为i，t+1时刻状态为j的概率值（ppt上的分子部分）
        for i in range(n):
            for j in range(n):
                tmp[i][j] = alpha[t][i] + A[i][j] + beta[t + 1][j] + B[j][fetch_index_by_obs_seq_f(Q, t + 1)]

        # 3.2 计算log_sum_exp的值(ppt上分母部分)
        lse = log_sum_exp(tmp.flat)

        # 3.3 计算最终的结果值
        for i in range(n):
            for j in range(n):
                ksi[t][i][j] = tmp[i][j] - lse

    # 4. 返回最终结果
    ksi


def baum_welch(pi, A, B, Q, max_iter=10, fetch_index_by_obs_seq=None):
    """
    根据传入的初始概率矩阵(pi、A、B)以及观测序列Q，使用baum_welch算法进行迭代求解最终的pi、A、B的值；最大迭代次数默认为10；最终更新结果保存在传入的参数矩阵中(pi\A\B)
    :param pi: 初始的状态随机概率矩阵, n*1 => 经过对数转换
    :param A:  状态转移矩阵, n*n => 经过对数转换
    :param B:  状态和观测值之间的转移矩阵, n*m => 经过对数转换
    :param Q:  观测值序列, 长度为T
    :param max_iter: 最大迭代次数，默认为10
    :param fetch_index_by_obs_seq: 根据序列获取对应的索引值，可以为None；是一个接受两个参数的函数，第一个参数为序列，第二个参数为索引值
    :return:  返回(pi, A, B)
    """
    # 1. 初始化
    fetch_index_by_obs_seq_f = fetch_index_by_obs_seq
    if fetch_index_by_obs_seq_f is None:
        # 默认是使用ord函数将对应位置的字符转换为ASCII码，eg: ord('a')=97; ord('中')=20013
        fetch_index_by_obs_seq_f = lambda obs, obs_index: ord(obs[obs_index])

    # 2. 初始化相关参数
    T = len(Q)
    n = np.shape(A)[0]
    if len(np.shape(B)) == 1:
        m = 1
    else:
        m = np.shape(B)[1]
    alpha_ = np.zeros((T, n))
    beta_ = np.zeros((T, n))
    gamma_ = np.zeros((T, n))
    ksi_ = np.zeros((T - 1, n, n))

    # 3. 开始遍历求解
    for time in range(max_iter):
        # 3.1 分别计算在当前情况下的alphe、beta、gamma、ksi的值
        calc_alpha(pi, A, B, Q, alpha_, fetch_index_by_obs_seq_f)
        calc_beta(pi, A, B, Q, beta_, fetch_index_by_obs_seq_f)
        calc_gamma(alpha_, beta_, gamma_)
        calc_ksi(alpha_, beta_, A, B, Q, ksi_, fetch_index_by_obs_seq_f)

        # 3.2 更新pi的值
        for i in range(n):
            pi[i] = gamma_[0][i]

        # 3.3 更新A的值
        tmp1 = np.zeros(T - 1)  # 对应ppt上分子
        tmp2 = np.zeros(T - 1)  # 对应ppt上的分母
        for i in range(n):
            for j in range(n):
                # 3.3.1 获取所有时刻从状态i转移到状态j的概率值
                for t in range(T - 1):
                    tmp1[t] = ksi_[t][i][j]
                    tmp2[t] = gamma_[t][i]

                # 3.3.2 计算状态转移矩阵
                A[i][j] = log_sum_exp(tmp1) - log_sum_exp(tmp2)

        # 3.4 更新B的值
        tmp1 = np.zeros(T)  # 对应ppt上分子
        tmp2 = np.zeros(T)  # 对应ppt上的分母
        for i in range(n):
            for o in range(m):
                # 3.4.1 获取所有时刻位于状态i的概率
                valid = 0
                for t in range(T):
                    # a. 计算分子的值
                    if o == fetch_index_by_obs_seq_f(Q, t):
                        # 当前观测值和对应观测值一致，计算分子
                        tmp1[valid] = gamma_[t][i]
                        valid += 1
                    # b. 累积分母的值
                    tmp2[t] = gamma_[t][i]

                # 3.4.2 更新状态转移矩阵的值B
                if valid == 0:
                    # 表示没有从i和o的观测值转移矩阵
                    B[i][o] = 0
                else:
                    B[i][o] = log_sum_exp(tmp1[:valid]) - log_sum_exp(tmp2)

    # 4. 返回最终结果
    return (pi, A, B)


def viterbi(pi, A, B, Q, fetch_index_by_obs_seq=None):
    """
    计算观测序列
    :param pi: 初始的状态随机概率矩阵, n*1 => 经过对数转换
    :param A:  状态转移矩阵, n*n => 经过对数转换
    :param B:  状态和观测值之间的转移矩阵, n*m => 经过对数转换
    :param Q:  观测值序列, 长度为T
    :param fetch_index_by_obs_seq: 根据序列获取对应的索引值，可以为None；是一个接受两个参数的函数，第一个参数为序列，第二个参数为索引值
    :return: 返回decode序列
    """
    # 1.初始化
    # 初始化序列转换为索引的方法
    fetch_index_by_obs_seq_f = fetch_index_by_obs_seq
    if fetch_index_by_obs_seq_f is None:
        fetch_index_by_obs_seq_f = lambda obs, obs_index: ord(obs[obs_index])

    # 2. 相关参数初始化
    T = len(Q)
    n = np.shape(A)[0]
    delta = np.zeros((T, n))
    # 存储的是上一个最优状态值,eg: pre_optima_index[t][i]表示时刻t对应状态为i，此时上一个时刻的最优状态（delta最大）为pre_optima_index[t][i]
    pre_optima_index = np.zeros((T, n), dtype=np.int)

    # 3 计算t=1时刻的delta值
    for i in range(n):
        delta[0][i] = pi[i] + B[i][fetch_index_by_obs_seq_f(Q, 0)]

    # 4 计算t=2、3、4.....时刻的delta值
    for t in range(1, T):
        for i in range(n):
            # 4.1 获取最大的值以及对应的最优索引位置
            max_delta = delta[t - 1][0] + A[0][i]
            optima_index = 0
            for j in range(1, n):
                tmp_delta = delta[t - 1][j] + A[j][i]
                if max_delta < tmp_delta:
                    max_delta = tmp_delta
                    optima_index = j

            # 4.2 计算最终的delta值以及最优索引位置
            delta[t][i] = max_delta + B[i][fetch_index_by_obs_seq_f(Q, t)]
            pre_optima_index[t][i] = optima_index

    # 5. 解码操作，查找到最大的结果值（回溯找最大的路径）
    decode = [-1 for i in range(T)]
    # 先找最后一个时刻的delta最大值
    max_delta_index = 0
    for i in range(1, n):
        if delta[T - 1][i] > delta[T - 1][max_delta_index]:
            max_delta_index = i
    decode[T - 1] = max_delta_index
    # 再根据转移的路径(最大转移路径), 找出最终的链路
    for t in range(T - 2, -1, -1):
        max_delta_index = pre_optima_index[t + 1][max_delta_index]
        decode[t] = max_delta_index

    # 6. 返回最终结果
    return decode

In [7]:

pi = np.array([0.2, 0.5, 0.3])
A = np.array([
    [0.5, 0.4, 0.1],
    [0.2, 0.2, 0.6],
    [0.2, 0.5, 0.3]
])
B = np.array([
    [0.4, 0.6],
    [0.8, 0.2],
    [0.5, 0.5]
])
# 对A\B\pi进行log转换、
pi = np.log(pi)
A = np.log(A)
B = np.log(B)

Q = ['白', '黑', '白', '白', '黑']
T = len(Q)
n = len(A)

print("测试前向概率计算....................")
alpha = np.zeros((T, n))
# 开始计算
calc_alpha(pi, A, B, Q, alpha, convert_obs_seq_2_index)
# 输出最终结果
# print(np.exp(alpha))
print(alpha)

# 计算最终概率值：
p = log_sum_exp(alpha[T - 1].flat)
print(Q, end="->出现的概率为:")
print(np.exp(p))

print("测试后向概率计算....................")
beta = np.zeros((T, n))
# 开始计算
calc_beta(pi, A, B, Q, beta, convert_obs_seq_2_index)
# 输出最终结果
# print(np.exp(beta))
print(beta)

# 计算最终概率值：
tmp_p = np.zeros(n)
for i in range(n):
    tmp_p[i] = pi[i] + B[i][convert_obs_seq_2_index(Q, 0)] + beta[0][i]
print("--------------")
p = log_sum_exp(tmp_p)
print(Q, end="->出现的概率为:")
print(np.exp(p))

print("测试gamma矩阵应用在最大概率预测的情况下.....................")
gamma = np.zeros((T, n))
calc_gamma(alpha, beta, gamma)
# print(np.exp(gamma))
print(gamma)
print("各个时刻最大概率的盒子为:", end='')
index = ['盒子1', '盒子2', '盒子3']
for p in gamma:
    print(index[p.tolist().index(np.max(p))], end="\t")
print()

print("测试ksi矩阵....................")
ksi = np.zeros((T - 1, n, n))
calc_ksi(alpha, beta, A, B, Q, ksi,convert_obs_seq_2_index)
print(ksi)

print("baum welch算法应用.......................")
baum_welch(pi, A, B, Q, max_iter=3, fetch_index_by_obs_seq=convert_obs_seq_2_index)
print("最终的pi矩阵：", end='')
print(np.exp(pi))
print("最终的状态转移矩阵：")
print(np.exp(A))
print("最终的状态-观测值转移矩阵：")
print(np.exp(B))

print("viterbi算法应用.....................")
state_seq = viterbi(pi, A, B, Q, convert_obs_seq_2_index)
print("最终结果为:", end='')
print(state_seq)
state = ['盒子1', '盒子2', '盒子3']
for i in state_seq:
    print(state[i], end='\t')

print("\n根据当前的状态值，预测未来的状态值....................")
### 假定当前状态为: 盒子1，那么接下来4次的状态分别是多少
### 假定当前状态为: 盒子1, 盒子2，那么接下来3次的状态分别是多少
### 假定当前状态为: 盒子1, 盒子3，那么接下来3次的状态分别是多少
### 假定当前状态为: 盒子1, 盒子1，那么接下来3次的状态分别是多少
current_state = [0, 0]
for t in range(1, 4):
    current_i = current_state[-1]
    max_ksi_j = ksi[t][current_i][0]
    max_j = 0
    for j in range(1, n):
        tmp = ksi[t][current_i][j]
        if tmp > max_ksi_j:
            max_ksi_j = tmp
            max_j = j

    current_state.append(max_j)
print(current_state)

测试前向概率计算....................
[[-2.52572864 -0.91629073 -1.89711998]
 [-2.40794561 -3.28608457 -1.92072985]
 [-3.4200133  -2.37103525 -3.27822782]
 [-4.07285395 -3.20676743 -3.34372927]
 [-4.25425309 -5.03406353 -4.00121617]]
['白', '黑', '白', '白', '黑']->出现的概率为:0.03900936690000001
测试后向概率计算....................
[[-2.83594215 -2.71456226 -2.95360138]
 [-2.00905764 -1.98525199 -1.90501104]
 [-1.37951738 -1.51868355 -1.2949922 ]
 [-0.84397007 -0.77652879 -0.99425227]
 [ 0.          0.          0.        ]]
--------------
['白', '黑', '白', '白', '黑']->出现的概率为:0.03900936690000001
测试gamma矩阵应用在最大概率预测的情况下.....................
[[-2.11771731 -0.38689951 -1.60676788]
 [-1.17304976 -2.02738308 -0.5817874 ]
 [-1.55557719 -0.64576532 -1.32926654]
 [-1.67287053 -0.73934273 -1.09402805]
 [-1.01029961 -1.79011005 -0.75726269]]
各个时刻最大概率的盒子为:盒子2	盒子3	盒子2	盒子2	盒子3	
测试ksi矩阵....................
[[[-2.4948056  -3.79275579 -4.18251847]
  [-1.80165842 -2.87646506 -0.78132109]
  [-2.78248767 -2.94100358 -2.45529752]]

 [[

In [8]:
"""训练分词用的HMM"""

import numpy as np
import math
import random


random.seed(28)
infinite = float(-2 ** 31)


def log_normalize(a):
    s = 0
    for i in a:
        s += i
    s = math.log(s)
    for i in range(len(a)):
        if a[i] == 0:
            a[i] = infinite
        else:
            a[i] = math.log(a[i]) - s


def fit(train_file_path, mode='r', encoding='utf-8'):
    """
    进行模型训练，并返回pi、A、B
    :param train_file_path:
    :return:
    """
    # 1. 加载数据
    with open(train_file_path, mode=mode, encoding=encoding) as reader:
        # 读取所有数据（因为数据格式第一个字符是不可见字符<文件描述符>）
        sentence = reader.read()[1:]

    # 1. 初始化pi、A、B
    pi = np.zeros(4)
    A = np.zeros((4, 4))
    B = np.zeros((4, 65536))

    # 2. 模型训练（使用MLE来预测） 0B/1M/2E/3S
    tokens = sentence.split(' ')
    last_i = 2  # 上一个词结束的状态
    for k, token in enumerate(tokens):
        token = token.strip()
        n = len(token)
        if n <= 0:
            continue

        if n == 1:
            pi[3] += 1
            A[last_i][3] += 1
            B[3][ord(token[0])] += 1
            last_i = 3
            continue

        # 初始化向量
        pi[0] += 1  # 作为开始
        pi[2] += 1  # 作为结束
        pi[1] += (n - 2)  # 中间词数目

        # 转移矩阵
        A[last_i][0] += 1
        last_i = 2
        if n == 2:
            A[0][2] += 1
        else:
            A[0][1] += 1
            A[1][1] += (n - 3)
            A[1][2] += 1

        # 发射矩阵
        B[0][ord(token[0])] += 1
        B[2][ord(token[n - 1])] += 1
        for i in range(1, n - 1):
            B[1][ord(token[i])] += 1

    # 正则化
    log_normalize(pi)
    for i in range(4):
        log_normalize(A[i])
        log_normalize(B[i])

    # 结果返回
    return pi, A, B


def dump(pi, A, B):
    """
    模型保存
    :param pi:
    :param A:
    :param B:
    :return:
    """
    n, m = np.shape(B)

    # 1. pi输出
    with open("pi.txt", "w") as f_pi:
        f_pi.write(str(n))
        f_pi.write('\n')
        f_pi.write(' '.join(map(str, pi)))

    # 2. A输出
    with open('A.txt', 'w') as f_a:
        f_a.write(str(n))
        f_a.write('\n')
        for a in A:
            f_a.write(' '.join(map(str, a)))
            f_a.write('\n')

    # 3. B输出
    with open('B.txt', 'w') as f_b:
        f_b.write(str(n))
        f_b.write('\n')
        f_b.write(str(m))
        f_b.write('\n')
        for b in B:
            f_b.write(' '.join(map(str, b)))
            f_b.write('\n')


def load():
    """
    模型加载
    :return:
    """
    with open('pi.txt', 'r', encoding='utf-8') as f_pi:
        f_pi.readline()  # 第一行不需要
        line = f_pi.readline()
        pi = list(map(float, line.strip().split(' ')))

    with open('A.txt', 'r', encoding='utf-8') as f_a:
        n = int(f_a.readline())
        A = np.zeros((n, n))
        i = 0
        for line in f_a:
            j = 0
            for v in map(float, line.strip().split(' ')):
                A[i][j] = v
                j += 1
            i += 1

    with open('B.txt', 'r', encoding='utf-8') as f_b:
        n = int(f_b.readline())
        m = int(f_b.readline())
        B = np.zeros((n, m))
        i = 0
        for line in f_b:
            j = 0
            for v in map(float, line.strip().split(' ')):
                B[i][j] = v
                j += 1
            i += 1

    return pi, A, B


def segment(sentence, decode):
    """
    分词
    :param sentence:
    :param decode:
    :return:
    """
    T = len(sentence)
    i = 0
    while i < T:  # B/M/E/S
        if decode[i] == 0 or decode[i] == 1:  # Begin
            j = i + 1
            while j < T:
                if decode[j] == 2:
                    break
                j += 1
            print(sentence[i:j + 1], end=' | ')
        elif decode[i] == 3 or decode[i] == 2:  # single
            print(sentence[i:i + 1], end=' | ')
        else:
            print("Error")
        i += 1





In [9]:
# 1. 模型训练
# 该数据默认是四个状态"B/M/E/S"
# B:begin, M:middle, E:end, S:single
# 分别代表每个状态代表的是该字在词语中的位置，B代表该字是词语中的起始字，M代表是词语中的中间字，E代表是词语中的结束字，S则代表是单字成词。
pi, A, B = fit('pku_training.utf8')
print(pi)
print(A)
print(B)

[-1.13813028 -2.63283402 -1.13813028 -1.24726168]
[[-2.14748365e+09 -1.91884919e+00 -1.58732900e-01 -2.14748365e+09]
 [-2.14748365e+09 -1.06226695e+00 -4.24145455e-01 -2.14748365e+09]
 [-7.20476004e-01 -2.14748365e+09 -2.14748365e+09 -6.66545397e-01]
 [-5.57413705e-01 -2.14748365e+09 -2.14748365e+09 -8.50241534e-01]]
[[-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]
 [-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]
 [-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]
 [-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]]


In [10]:
# 2. 模型输出
dump(pi, A, B)

In [14]:



# # 2. 模型输出
# dump(pi, A, B)

# 3. 模型加载
pi, A, B = load()
print(pi)
print(A)
print(B)



[-1.1381302786652014, -2.632834016555254, -1.1381302786652014, -1.247261683317193]
[[-2.14748365e+09 -1.91884919e+00 -1.58732900e-01 -2.14748365e+09]
 [-2.14748365e+09 -1.06226695e+00 -4.24145455e-01 -2.14748365e+09]
 [-7.20476004e-01 -2.14748365e+09 -2.14748365e+09 -6.66545397e-01]
 [-5.57413705e-01 -2.14748365e+09 -2.14748365e+09 -8.50241534e-01]]
[[-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]
 [-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]
 [-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]
 [-2.14748365e+09 -2.14748365e+09 -2.14748365e+09 ... -2.14748365e+09
  -2.14748365e+09 -2.14748365e+09]]


In [15]:
# 4. 进行分词操作
with open('novel.txt', 'r', encoding='utf-8') as reader:
    data = reader.read()[1:]
decode = viterbi(pi, A, B, data)
segment(data, decode)

美 | 关系 | 系 | 持续 | 续 | 交恶 | 恶 | 的 | 大 | 背景 | 景 | 下 | ， | 外界 | 界 | 的 | 目光 | 光 | 都 | 聚焦 | 焦 | 在 | 中国 | 国 | 如何 | 何 | 保持 | 持 | 发展 | 展 | ， | 尤其 | 其 | 是 | 近年 | 年 | 来 | 被 | 反复 | 复 | 提及的2035年 | 及的2035年 | 的2035年 | 2035年 | 035年 | 35年 | 5年 | 年 | 远景 | 景 | 目标 | 标 | ， | 届 | 时 | 中国 | 国 | 经济 | 济 | 是 | 否会 | 会 | 全面 | 面 | 超越 | 越 | 美国 | 国 | ？ | 以及 | 及 | 为 | 了 | 达到 | 到 | 这 | 一 | 目标 | 标 | ， | 中国 | 国 | 面临 | 临 | 哪些 | 些 | 困难 | 难 | 和 | 挑战 | 战 | ？ | 